# Setup

## Imports

In [2]:
import time
import numpy as np
import pytest
from JuliaSet import calculate_z_serial_purepython
from test_juliaset import test_calc_pure_python


## Julia Set Values

In [3]:
# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# Exercise 1: PyTest with the Julia Set Code

## 1.1: Testing with PyTest Framework

### Implemented Test

In [11]:
def test_calc_pure_python(desired_width=1000, max_iterations=300):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    print("new sum: ", sum(output))
    assert sum(output) == 33219980

### Test Results

In [4]:
! pytest test_juliaset.py

============================= test session starts ==============================
platform darwin -- Python 3.11.6, pytest-8.3.4, pluggy-1.5.0
rootdir: /Users/carlottaholzle/Desktop/KTH Assignments/data-structures-methods
collected 1 item                                                               

test_juliaset.py .                                                       [100%]

============================== 1 passed in 2.79s ===============================


## 1.2: Testing with Varying Iterations & Grid Points

To test with varying iterations and grid points, we would use a parameterized test. This allows us to assert different expected values based on the parameters and test each of them.

### Implemented Test

In [5]:
import pytest
@pytest.mark.parameterize('desired_width, max_iterations, expected', [(1000, 300, 33219980)])
def test_calc_pure_python_param(desired_width, max_iterations, expected):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    print("new sum: ", sum(output))
    assert sum(output) == expected

### Test Results

In [6]:
! pytest test_juliaset_param.py

============================= test session starts ==============================
platform darwin -- Python 3.11.6, pytest-8.3.4, pluggy-1.5.0
rootdir: /Users/carlottaholzle/Desktop/KTH Assignments/data-structures-methods
collected 1 item                                                               

test_juliaset_param.py .                                                 [100%]

============================== 1 passed in 2.91s ===============================


# Exercise 2: Python DGEMM Benchmark Operation

## 2.1: Implementing DGEMM with NumPy

In [7]:
a_np = np.ones((3, 3), dtype=np.double)
b_np = np.ones((3, 3), dtype=np.double)
c_np = np.ones((3, 3), dtype=np.double)
expected_np = np.array([[4,4,4],[4,4,4],[4,4,4]])

def dgemm_numpy(a, b, c):
  N = a.shape[0]
  for i in range(N):
    for j in range(N):
      for k in range(N):
        c[i][j] = c[i][j] + a[i][k] * b[k][j]
  return c

## 2.2: Unit Test for DGEMM

### Test

In [8]:
def test_dgemm():
  assert np.array_equal(dgemm_numpy(a_np,b_np,c_np), expected_np) == True

### Test Results

In [9]:
! pytest test_dgemm.py

============================= test session starts ==============================
platform darwin -- Python 3.11.6, pytest-8.3.4, pluggy-1.5.0
rootdir: /Users/carlottaholzle/Desktop/KTH Assignments/data-structures-methods
collected 1 item                                                               

test_dgemm.py .                                                          [100%]

============================== 1 passed in 0.05s ===============================


In [18]:
import numpy as np
import time

def dgemm_numpy(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

def measure_execution_time(matrix_size, runs=5):
    times = []
    for _ in range(runs):
        # Initialize matrices with random values
        a = np.ones((matrix_size, matrix_size), dtype=np.double)
        b = np.ones((matrix_size, matrix_size), dtype=np.double)
        c = np.zeros((matrix_size, matrix_size), dtype=np.double)

        # Measure execution time
        start_time = time.time()
        dgemm_numpy(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)

    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Testing with different matrix sizes
matrix_sizes = [10, 50, 100, 200]  # You can increase these sizes for larger benchmarks
results = []

for size in matrix_sizes:
    avg_time, std_dev = measure_execution_time(size)
    results.append((size, avg_time, std_dev))

# Print results
print("Matrix Size | Avg. Time (s) | Std. Dev. (s)")
print("------------------------------------------")
for size, avg, std in results:
    print(f"{size:^11} | {avg:^13.6f} | {std:^12.6f}")

Matrix Size | Avg. Time (s) | Std. Dev. (s)
------------------------------------------
    10      |   0.000995    |   0.000230  
    50      |   0.050996    |   0.006715  
    100     |   0.379782    |   0.021228  
    200     |   2.934099    |   0.062459  


Implementation with array: 

In [12]:
import array

def dgemm_array(a, b, c, N):
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Helper function to create 2D arrays
def create_2d_array(size):
    return [array.array('d', [0] * size) for _ in range(size)]

Implementation using list 

In [13]:
def dgemm_list(a, b, c):
    N = len(a)
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

## Performance Measurement Code LIST, ARRAY, NumPy

The code handles and benchmarks multiple implementations sequentially in the same script. If the system runs other background processes or has load variations during the benchmarking process, these might disproportionately affect one implementation’s results. Furthermore, timing overhead might be added due to the use of lambdas for the array implementation. 

In [20]:
import numpy as np
import time

# Measure execution time for different implementations
def measure_time_implementation(implementation, matrix_size, runs=5):
    times = []
    for _ in range(runs):
        if implementation == "list":
            # Initialize matrices as lists
            a = [[1.0] * matrix_size for _ in range(matrix_size)]
            b = [[1.0] * matrix_size for _ in range(matrix_size)]
            c = [[0.0] * matrix_size for _ in range(matrix_size)]
            func = dgemm_list
        elif implementation == "array":
            # Initialize matrices as arrays
            a = create_2d_array(matrix_size)
            b = create_2d_array(matrix_size)
            c = create_2d_array(matrix_size)
            for i in range(matrix_size):
                for j in range(matrix_size):
                    a[i][j] = b[i][j] = 1.0
            func = lambda a, b, c: dgemm_array(a, b, c, matrix_size)
        elif implementation == "numpy":
            # Initialize matrices as NumPy arrays
            a = np.ones((matrix_size, matrix_size), dtype=np.double)
            b = np.ones((matrix_size, matrix_size), dtype=np.double)
            c = np.zeros((matrix_size, matrix_size), dtype=np.double)
            func = dgemm_numpy
        else:
            raise ValueError("Unsupported implementation")

        # Measure time
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)

    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Compare implementations
implementations = ["list", "array", "numpy"]
matrix_sizes = [10, 50, 100, 200, 400]  # You can extend these sizes as needed
results = []

for size in matrix_sizes:
    for impl in implementations:
        avg_time, std_dev = measure_time_implementation(impl, size)
        results.append((impl, size, avg_time, std_dev))

# Print results
print("Implementation | Matrix Size | Avg. Time (s) | Std. Dev. (s)")
print("-----------------------------------------------------------")
for impl, size, avg, std in results:
    print(f"{impl:^13} | {size:^11} | {avg:^13.6f} | {std:^12.6f}")

Implementation | Matrix Size | Avg. Time (s) | Std. Dev. (s)
-----------------------------------------------------------
    list      |     10      |   0.000101    |   0.000005  
    array     |     10      |   0.000199    |   0.000057  
    numpy     |     10      |   0.000768    |   0.000141  
    list      |     50      |   0.015739    |   0.008049  
    array     |     50      |   0.010727    |   0.000150  
    numpy     |     50      |   0.047675    |   0.001315  
    list      |     100     |   0.048073    |   0.001517  
    array     |     100     |   0.085522    |   0.001029  
    numpy     |     100     |   0.372809    |   0.003653  
    list      |     200     |   0.384661    |   0.020841  
    array     |     200     |   0.668303    |   0.001794  
    numpy     |     200     |   2.929246    |   0.068054  
    list      |     400     |   3.115510    |   0.021850  
    array     |     400     |   5.645596    |   0.049853  
    numpy     |     400     |   24.057626   |   0.557

How does the computational performance, e.g., the std, vary with increasing the size of the matrices, and why so?

### 1. Average Time Increases with Matrix Size:
As the matrix size increases, the average execution time grows significantly for all implementations (list, array, and NumPy).
This is expected because the computational complexity of matrix multiplication is $O(N^3)$, meaning the number of operations grows cubically with the matrix size N.
### 2.Standard Deviation Trends:
For small matrices (e.g., 10x10), the std. dev. is relatively small for all implementations.
As the matrix size increases, the std. dev. generally increases, but the rate of increase varies across implementations.
NumPy exhibits a higher std. dev. compared to lists and arrays, especially for larger matrices (e.g., 400x400).
### 3. Relative Performance:
For small matrices, the list implementation is the fastest, followed by arrays, and then NumPy. This trend stays consistent over increasing matrix size. 
For larger matrices, NumPy becomes significantly slower compared to lists and arrays, despite being optimized for numerical computations. This is likely due to NumPy beeing optimized for vectorized operations, but the implementation of DGEMM in this exercise uses explicit Python loops (for i, for j, for k). This negates the benefits of NumPy's vectorization and introduces significant overhead.

# Task 2.4 
To calculate the FLOPS/s (floating-point operations per second), we first need to determine the number of floating-point operations (FLOPs) performed in the DGEMM operation. Then we divide the total number of FLOPs by the average time taken.

In DGEMM, the operation is: 
$C[i][j] = C[i][j] + A[i][k] \times B[k][j] $
- The operation involves:
	- 1 multiplication:  A[i][k] \times B[k][j] 
	- 1 addition:  C[i][j] + \text{result of multiplication} 
	- Total FLOPs per iteration of the innermost loop: 2 FLOPs

The total number of iterations across the nested loops is $N^3$, where $N$ is the matrix size. Therefore:

$\text{Total FLOPs} = 2 \times N^3$


The FLOPS/s for a given implementation is:

$\text{FLOPS/s} = \frac{\text{Total FLOPs}}{\text{Avg. Time (s)}}$


In [21]:
def calculate_flops(matrix_size, avg_time):
    # Calculate the total number of FLOPs
    total_flops = 2 * (matrix_size ** 3)
    # Calculate FLOPS/s
    flops_per_second = total_flops / avg_time
    return total_flops, flops_per_second

# Timing results (replace these with your values)
results = [
    ("list", 10, 0.000101),
    ("array", 10, 0.000199),
    ("numpy", 10, 0.000768),
    ("list", 50, 0.015739),
    ("array", 50, 0.010727),
    ("numpy", 50, 0.047675),
    ("list", 100, 0.048073),
    ("array", 100, 0.085522),
    ("numpy", 100, 0.372809),
    ("list", 200, 0.384661),
    ("array", 200, 0.668303),
    ("numpy", 200, 2.929246),
    ("list", 400, 3.115510),
    ("array", 400, 5.645596),
    ("numpy", 400, 24.057626),
]

# Calculate FLOPS/s for each result
print("Implementation | Matrix Size | Total FLOPs  | FLOPS/s")
print("---------------------------------------------------------")
for impl, size, avg_time in results:
    total_flops, flops_per_second = calculate_flops(size, avg_time)
    print(f"{impl:^13} | {size:^11} | {total_flops:^11} | {flops_per_second:.2e}")

Implementation | Matrix Size | Total FLOPs  | FLOPS/s
---------------------------------------------------------
    list      |     10      |    2000     | 1.98e+07
    array     |     10      |    2000     | 1.01e+07
    numpy     |     10      |    2000     | 2.60e+06
    list      |     50      |   250000    | 1.59e+07
    array     |     50      |   250000    | 2.33e+07
    numpy     |     50      |   250000    | 5.24e+06
    list      |     100     |   2000000   | 4.16e+07
    array     |     100     |   2000000   | 2.34e+07
    numpy     |     100     |   2000000   | 5.36e+06
    list      |     200     |  16000000   | 4.16e+07
    array     |     200     |  16000000   | 2.39e+07
    numpy     |     200     |  16000000   | 5.46e+06
    list      |     400     |  128000000  | 4.11e+07
    array     |     400     |  128000000  | 2.27e+07
    numpy     |     400     |  128000000  | 5.32e+06


### Comparing to Theoretical Peak Performance

Substituding values for Mac Pro 2020 M1 chip: 
	•	Clock Frequency = 3.2 GHz = 3.2 × 10⁹ cycles/second.
	•	FLOPs per Cycle = 2 (double precision).
	•	Number of High-Performance Cores = 4.

$\text{Peak FLOPS/s} = 3.2 \times 10^9 \times 2 \times 4 = 25.6 \, \text{GFLOPS (double precision)} $

For single precision (32-bit floating-point), NEON can handle 4 single-precision numbers per cycle per core:

$\text{Peak FLOPS/s (single precision)} = 3.2 \times 10^9 \times 4 \times 4 = 51.2 \, \text{GFLOPS}$


The theoretical peak performance of the Apple M1 chip at 25.6 GFLOPS (double precision) far exceeds the actual measured performance, indicating that real-world DGEMM implementations face significant bottlenecks, such as memory bandwidth limitations, system overhead, and non-ideal utilization of vectorized instructions. This gap highlights the challenges in achieving the full computational potential of the hardware, as practical workloads rarely operate at maximum efficiency due to these constraints.


In [24]:
import numpy as np
import time

# Custom DGEMM implementation
def dgemm_custom(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Function to measure execution time
def measure_execution_time(func, a, b, c, runs=5):
    times = []
    for _ in range(runs):
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)
    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Function to benchmark both implementations
def benchmark(matrix_size, runs=5):
    # Initialize matrices
    a = np.random.rand(matrix_size, matrix_size).astype(np.double)
    b = np.random.rand(matrix_size, matrix_size).astype(np.double)
    c_custom = np.zeros((matrix_size, matrix_size), dtype=np.double)
    c_numpy = np.zeros((matrix_size, matrix_size), dtype=np.double)

    # Measure custom DGEMM
    avg_time_custom, std_dev_custom = measure_execution_time(dgemm_custom, a, b, c_custom, runs)

    # Measure NumPy matmul
    avg_time_numpy, std_dev_numpy = measure_execution_time(lambda x, y, z: np.matmul(x, y, out=z), a, b, c_numpy, runs)

    # Print results
    print(f"Matrix Size: {matrix_size}x{matrix_size}")
    print(f"Custom DGEMM - Avg. Time: {avg_time_custom:.6f}s, Std. Dev.: {std_dev_custom:.6f}s")
    print(f"NumPy matmul - Avg. Time: {avg_time_numpy:.6f}s, Std. Dev.: {std_dev_numpy:.6f}s")
    print()

# Run benchmark for different matrix sizes
matrix_sizes = [10, 50, 100, 200]
for size in matrix_sizes:
    benchmark(size)

Matrix Size: 10x10
Custom DGEMM - Avg. Time: 0.000628s, Std. Dev.: 0.000013s
NumPy matmul - Avg. Time: 0.000009s, Std. Dev.: 0.000009s

Matrix Size: 50x50
Custom DGEMM - Avg. Time: 0.056542s, Std. Dev.: 0.018030s
NumPy matmul - Avg. Time: 0.000012s, Std. Dev.: 0.000016s

Matrix Size: 100x100
Custom DGEMM - Avg. Time: 0.376978s, Std. Dev.: 0.001663s
NumPy matmul - Avg. Time: 0.000014s, Std. Dev.: 0.000005s

Matrix Size: 200x200
Custom DGEMM - Avg. Time: 2.949500s, Std. Dev.: 0.037196s
NumPy matmul - Avg. Time: 0.000068s, Std. Dev.: 0.000018s



In [23]:
import numpy as np
import time

# Custom DGEMM implementation
def dgemm_custom(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Function to measure execution time
def measure_execution_time(func, a, b, c, runs=5):
    times = []
    for _ in range(runs):
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)
    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Function to calculate FLOPS
def calculate_flops(matrix_size, avg_time):
    total_flops = 2 * (matrix_size ** 3)  # 2 FLOPs per iteration (1 multiply + 1 add)
    flops_per_second = total_flops / avg_time if avg_time > 0 else 0
    return total_flops, flops_per_second

# Function to benchmark both implementations and compute FLOPS
def benchmark(matrix_size, runs=5):
    # Initialize matrices
    a = np.random.rand(matrix_size, matrix_size).astype(np.double)
    b = np.random.rand(matrix_size, matrix_size).astype(np.double)
    c_custom = np.zeros((matrix_size, matrix_size), dtype=np.double)
    c_numpy = np.zeros((matrix_size, matrix_size), dtype=np.double)

    # Measure custom DGEMM
    avg_time_custom, std_dev_custom = measure_execution_time(dgemm_custom, a, b, c_custom, runs)
    total_flops_custom, flops_custom = calculate_flops(matrix_size, avg_time_custom)

    # Measure NumPy matmul
    avg_time_numpy, std_dev_numpy = measure_execution_time(lambda x, y, z: np.matmul(x, y, out=z), a, b, c_numpy, runs)
    total_flops_numpy, flops_numpy = calculate_flops(matrix_size, avg_time_numpy)

    # Print results
    print(f"Matrix Size: {matrix_size}x{matrix_size}")
    print(f"Custom DGEMM - Avg. Time: {avg_time_custom:.6f}s, Std. Dev.: {std_dev_custom:.6f}s, FLOPS: {flops_custom:.2e}")
    print(f"NumPy matmul - Avg. Time: {avg_time_numpy:.6f}s, Std. Dev.: {std_dev_numpy:.6f}s, FLOPS: {flops_numpy:.2e}")
    print()

# Run benchmark for different matrix sizes
matrix_sizes = [10, 50, 100, 200]
for size in matrix_sizes:
    benchmark(size)

Matrix Size: 10x10
Custom DGEMM - Avg. Time: 0.000797s, Std. Dev.: 0.000121s, FLOPS: 2.51e+06
NumPy matmul - Avg. Time: 0.000004s, Std. Dev.: 0.000004s, FLOPS: 4.51e+08

Matrix Size: 50x50
Custom DGEMM - Avg. Time: 0.050790s, Std. Dev.: 0.004178s, FLOPS: 4.92e+06
NumPy matmul - Avg. Time: 0.000027s, Std. Dev.: 0.000024s, FLOPS: 9.35e+09

Matrix Size: 100x100
Custom DGEMM - Avg. Time: 0.390325s, Std. Dev.: 0.015579s, FLOPS: 5.12e+06
NumPy matmul - Avg. Time: 0.000018s, Std. Dev.: 0.000012s, FLOPS: 1.11e+11

Matrix Size: 200x200
Custom DGEMM - Avg. Time: 2.960337s, Std. Dev.: 0.038274s, FLOPS: 5.40e+06
NumPy matmul - Avg. Time: 0.000075s, Std. Dev.: 0.000019s, FLOPS: 2.14e+11



NumPy’s matmul not only leverages the full capabilities of the M1 chip but also surpasses the theoretical peak due to highly optimized, hardware-specific operations. This demonstrates that for computationally intensive tasks, using optimized libraries like BLAS is critical for achieving performance close to or beyond theoretical limits.